In [ ]:
from torch.utils.data import DataLoader
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, InputExample, models, evaluation, losses
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

# Obtendo os dados

In [ ]:
# Lendo e salvando para CSV
def obter_dados_csv():
    df_dados = pd.read_csv('dados.csv')
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados

In [ ]:
df_dados = obter_dados_csv()
df_dados

# Removendo os dígitos

In [ ]:
df_dados["curriculos"] = df_dados["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_dados["vagas"] = df_dados["vagas"].apply(lambda x: re.sub('\d+', '', x))

# Transformando as notas em scores de similaridade

In [ ]:
min_max_scaler = MinMaxScaler()
min_max_scaler.set_output(transform='pandas');

In [ ]:
df_dados["notas"] = min_max_scaler.fit_transform(df_dados["notas"].values.reshape(-1, 1))
# 1 -> 0.00
# 2 -> 0.25
# 3 -> 0.50
# 4 -> 0.75
# 5 -> 1.00
df_dados

# Preparando os dados para treinamento

In [ ]:
data_examples = []
for index, row in df_dados.iterrows():
    data_examples.append(InputExample(texts=[row['curriculos'], row['vagas']], label=row['notas']))

# Dividindo os dados em 80% treino, 20% validação e 20% para testes
data_examples = shuffle(data_examples, random_state=42)
indice_treino = int(len(data_examples) * 0.6)
indice_val = int(len(data_examples) * 0.2)

train_examples = data_examples[:indice_treino]
val_examples = data_examples[indice_treino:indice_treino+indice_val]
test_examples = data_examples[indice_treino+indice_val:]

# Criando os Dataloaders do PyTorch
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

# Treinamento

In [ ]:
checkpoint = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

word_embedding_model = models.Transformer(checkpoint, cache_dir=f'model/{checkpoint}')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_loss = losses.CosineSimilarityLoss(model)

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='sbert')

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, evaluator=evaluator, show_progress_bar=True, output_path=f'model_FT/{checkpoint}')

# Testando um sistema de recomendação de vagas para um currículo

test_curriculo = 'Nome: Laura Costa - Objetivo: Busco uma posição como Analista Econômico, onde posso aplicar minha formação acadêmica em Economia e aprimorar minhas habilidades em análise econômica. Formação Acadêmica: Bacharelado em Economia - Universidade Federal de Estado Y (-) Experiência Profissional: Assistente de Análise Econômica - Empresa de Consultoria Econômica LTDA - Cidade Financeira, Estado Y (-Presente) Coleta de dados econômicos. Auxílio na elaboração de relatórios e análises. Habilidades: Conhecimentos intermediários em análise econômica. Familiaridade com ferramentas como Excel e SPSS. Idiomas: Inglês: Avançado Espanhol: Básico'

test_vagas = list(set([test_example.texts[1] for test_example in test_examples]))

In [ ]:
embedding_curriculos = model.encode(test_curriculo)
embedding_vagas = [model.encode(vaga) for vaga in test_vagas]
scores_similaridade = util.cos_sim(embedding_curriculos, embedding_vagas)

In [ ]:
# Encontrando os pares curriculo-vaga com as maiores similaridades de cosseno
pares = []
for index, score in enumerate(scores_similaridade[0]):
    pares.append({"index": index, "score": score})

# Ordena os pares pelos scores na ordem decrescente
pares = sorted(pares, key=lambda x: x["score"], reverse=True)

In [ ]:
print(f' Currículo: {test_curriculo} \n\n')
for par in pares[0:5]:
    print(f' Vaga: {test_vagas[par["index"]]} \n Similaridade predita após o ajuste fino: {par["score"]} \n')